In [1]:
import os, sys
sys.path.append("..")

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 

import numpy as np
import torch
import torch.nn as nn
import torchvision
import gc

from src.tools import unfreeze, freeze
from src.tools import load_dataset, get_loader_stats

from copy import deepcopy
import json

from tqdm import tqdm_notebook as tqdm
from IPython.display import clear_output

# This needed to use dataloaders for some datasets
from PIL import PngImagePlugin
LARGE_ENOUGH_NUMBER = 100
PngImagePlugin.MAX_TEXT_CHUNK = LARGE_ENOUGH_NUMBER * (1024**2)

from src.anime_benchmark import get_anime_benchmark

In [2]:
gc.collect(); torch.cuda.empty_cache()

## Main Config

In [3]:
DEVICE_ID = 3
EPS = 0.1
A = 1
N_EPOCHS = 1

assert torch.cuda.is_available()
torch.cuda.set_device(f'cuda:{DEVICE_ID}')

In [5]:
benchmark = get_anime_benchmark(batch_size=20, eps=EPS, A=A, glow_device="cuda", samples_device="cuda", download=False)

../src/model.py:102: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  w_s = torch.from_numpy(w_s)


In [6]:
X_sampler = benchmark.X_sampler
X_test_sampler = benchmark.X_test_sampler

Y_sampler = benchmark.Y_sampler
Y_test_sampler = benchmark.Y_test_sampler

In [7]:
for sampler in [Y_test_sampler]:
    mu, sigma = get_loader_stats(sampler.loader, n_epochs=N_EPOCHS)
    print('Trace of sigma: {}'.format(np.trace(sigma)))
    stats = {'mu' : mu.tolist(), 'sigma' : sigma.tolist()}
    print('Stats computed')

    filename = f"Anime_Y_test_eps_{EPS}_A_{A}.json"
    with open(filename, 'w') as fp:
        json.dump(stats, fp)
    print('States saved to {}'.format(filename))

Trace of sigma: 77.77547948083821
Stats computed
States saved to Anime_Y_test_eps_0.1_A_1
